In [103]:
import numpy as np
import pandas as pd
import requests as rq
import matplotlib.pyplot as plt

In [104]:
data_url = "http://datapoint.metoffice.gov.uk/public/data/val/wxobs/all/json/3772?res=hourly&key=12a3f254-6c4a-4f6d-a464-a7a2290a7226"

In [105]:
response = rq.get(data_url)

In [106]:
data =response.json()

In [107]:
weather_types = {
    'NA': 'Not available',
    '-1': 'Trace rain',
    '0': 'Clear night',
    '1': 'Sunny day',
    '2': 'Partly cloudy (night)',
    '3': 'Partly cloudy (day)',
    '4': 'Not used',
    '5': 'Mist',
    '6': 'Fog',
    '7': 'Cloudy',
    '8': 'Overcast',
    '9': 'Light rain shower (night)',
    '10': 'Light rain shower (day)',
    '11': 'Drizzle',
    '12': 'Light rain',
    '13': 'Heavy rain shower (night)',
    '14': 'Heavy rain shower (day)',
    '15': 'Heavy rain',
    '16': 'Sleet shower (night)',
    '17': 'Sleet shower (day)',
    '18': 'Sleet',
    '19': 'Hail shower (night)',
    '20': 'Hail shower (day)',
    '21': 'Hail',
    '22': 'Light snow shower (night)',
    '23': 'Light snow shower (day)',
    '24': 'Light snow',
    '25': 'Heavy snow shower (night)',
    '26': 'Heavy snow shower (day)',
    '27': 'Heavy snow',
    '28': 'Thunder shower (night)',
    '29': 'Thunder shower (day)',
    '30': 'Thunder'
}

In [108]:
pd.DataFrame(data['SiteRep']['Wx']['Param'])

,name,units,$
0,G,mph,Wind Gust
1,T,C,Temperature
2,V,m,Visibility
3,D,compass,Wind Direction
4,S,mph,Wind Speed
5,W,,Weather Type
6,P,hpa,Pressure
7,Pt,Pa/s,Pressure Tendency
8,Dp,C,Dew Point
9,H,%,Screen Relative Humidity


In [109]:
visibility_dict = {
    'UN': 'Unknown',
    'VP': 'Very poor - Less than 1 km',
    'PO': 'Poor - Between 1-4 km',
    'MO': 'Moderate - Between 4-10 km',
    'GO': 'Good - Between 10-20 km',
    'VG': 'Very good - Between 20-40 km',
    'EX': 'Excellent - More than 40 km'
}

In [110]:
compass_directions_map = {
    'N': 1,
    'NNE':2,
    'NE': 3,
    'ENE':4,
    'E': 5,
    'ESE':6,
    'SE': 7,
    'SSE':8,
    'S': 9,
    'SSW':10,
    'SW': 11,
    'WSW':12,
    'W': 13,
    'WNW':14,
    'NW': 15,
    'NNW':16
}

In [111]:
Pressure_tendency_map = {
    'F': 0,
    'R': 1,
    'S': 2
}

In [112]:
headers = {'D':'Wind Direction(compass)', 'G':'Wind Gust(mph)', 'H':'Screen Relative Humidity(%)', 'P':'Pressure(hpa)', 'S':'Wind Speed(mph)', 'T':'Temperature(C)', 'V':'Visibility(m)', 'W':'Weather Type', 'Pt':'Pressure Tendency', 'Dp':'Dew Point(C)', '$':'Minutes Since 12o Clock'}

In [113]:
data['SiteRep']['DV']

{'dataDate': '2023-08-11T13:00:00Z',
 'type': 'Obs',
 'Location': {'i': '3772',
  'lat': '51.479',
  'lon': '-0.449',
  'name': 'HEATHROW',
  'country': 'ENGLAND',
  'continent': 'EUROPE',
  'elevation': '25.0',
  'Period': [{'type': 'Day',
    'value': '2023-08-10Z',
    'Rep': [{'D': 'S',
      'G': '17',
      'H': '56.3',
      'P': '1018',
      'S': '9',
      'T': '24.6',
      'V': '55000',
      'W': '8',
      'Pt': 'F',
      'Dp': '15.4',
      '$': '780'},
     {'D': 'S',
      'G': '17',
      'H': '56.8',
      'P': '1018',
      'S': '10',
      'T': '24.8',
      'V': '50000',
      'W': '8',
      'Pt': 'F',
      'Dp': '15.7',
      '$': '840'},
     {'D': 'S',
      'G': '17',
      'H': '56.0',
      'P': '1018',
      'S': '10',
      'T': '24.8',
      'V': '60000',
      'W': '8',
      'Pt': 'F',
      'Dp': '15.5',
      '$': '900'},
     {'D': 'S',
      'G': '21',
      'H': '53.2',
      'P': '1017',
      'S': '13',
      'T': '26.2',
      'V': '60000',
 

In [114]:
weather_data = None
for day in data['SiteRep']['DV']['Location']['Period']:
    df = pd.DataFrame(day['Rep'])
    df['date'] = day['value']
    weather_data = pd.concat([weather_data, df], ignore_index=True)

In [115]:
weather_data.rename(columns=headers, inplace=True)

In [116]:
weather_data['Wind Direction(compass)'] = weather_data['Wind Direction(compass)'].map(compass_directions_map)

In [117]:
weather_data['Pressure Tendency'] = weather_data['Pressure Tendency'].map(Pressure_tendency_map)

In [118]:
weather_data['date'] = pd.to_datetime(weather_data['date'], format='%Y-%m-%dZ')

In [119]:
column_datatypes = {'Wind Direction(compass)':int, 'Wind Gust(mph)':float,
       'Screen Relative Humidity(%)':float, 'Pressure(hpa)':float, 'Wind Speed(mph)':float,
       'Temperature(C)':float, 'Visibility(m)':float, 'Weather Type':int, 'Pressure Tendency':int,
       'Dew Point(C)':float, 'Minutes Since 12o Clock':int}

In [120]:
weather_data

,Wind Direction(compass),Wind Gust(mph),Screen Relative Humidity(%),Pressure(hpa),Wind Speed(mph),Temperature(C),Visibility(m),Weather Type,Pressure Tendency,Dew Point(C),Minutes Since 12o Clock,date
0,9,17,56.3,1018,9,24.6,55000,8,0,15.4,780,2023-08-10
1,9,17,56.8,1018,10,24.8,50000,8,0,15.7,840,2023-08-10
2,9,17,56.0,1018,10,24.8,60000,8,0,15.5,900,2023-08-10
3,9,21,53.2,1017,13,26.2,60000,1,0,16.0,960,2023-08-10
4,9,21,52.9,1017,11,25.1,55000,1,0,14.9,1020,2023-08-10
5,8,17,54.4,1017,13,24.1,55000,1,0,14.4,1080,2023-08-10
6,9,19,57.2,1017,10,23.6,50000,8,1,14.7,1140,2023-08-10
7,9,21,65.5,1018,9,21.7,45000,7,1,15.0,1200,2023-08-10
8,9,15,70.1,1018,9,20.5,45000,2,1,14.9,1260,2023-08-10
9,9,15,76.0,1018,9,19.4,40000,7,1,15.1,1320,2023-08-10


In [130]:
weather_data = weather_data.astype(column_datatypes)

In [132]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Wind Direction(compass)      25 non-null     int64         
 1   Wind Gust(mph)               25 non-null     float64       
 2   Screen Relative Humidity(%)  25 non-null     float64       
 3   Pressure(hpa)                25 non-null     float64       
 4   Wind Speed(mph)              25 non-null     float64       
 5   Temperature(C)               25 non-null     float64       
 6   Visibility(m)                25 non-null     float64       
 7   Weather Type                 25 non-null     int64         
 8   Pressure Tendency            25 non-null     int64         
 9   Dew Point(C)                 25 non-null     float64       
 10  Minutes Since 12o Clock      25 non-null     int64         
 11  date                         25 non-null     da

In [134]:
weather_data['day'] = weather_data['date'].dt.day
weather_data['month'] = weather_data['date'].dt.month
weather_data['year'] = weather_data['date'].dt.year

In [137]:
weather_data.drop('date', axis=1, inplace=True)

In [140]:
weather_data.head()

,Wind Direction(compass),Wind Gust(mph),Screen Relative Humidity(%),Pressure(hpa),Wind Speed(mph),Temperature(C),Visibility(m),Weather Type,Pressure Tendency,Dew Point(C),Minutes Since 12o Clock,day,month,year
0,9,17.0,56.3,1018.0,9.0,24.6,55000.0,8,0,15.4,780,10,8,2023
1,9,17.0,56.8,1018.0,10.0,24.8,50000.0,8,0,15.7,840,10,8,2023
2,9,17.0,56.0,1018.0,10.0,24.8,60000.0,8,0,15.5,900,10,8,2023
3,9,21.0,53.2,1017.0,13.0,26.2,60000.0,1,0,16.0,960,10,8,2023
4,9,21.0,52.9,1017.0,11.0,25.1,55000.0,1,0,14.9,1020,10,8,2023
